In [4]:
# Imports the necessary libraries.
import requests
import csv
from bs4 import BeautifulSoup
import json
import pandas as pd
import datetime

In [5]:
# URL of the website to be scraped
url = "https://books.toscrape.com/"

# Make a GET request to fetch the raw HTML content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, "lxml")

# Get all the book categories from the sidebar
categories = soup.find('div', class_='side_categories').find_all('a')
category_urls = [url + category['href'] for category in categories[1:]]
# Adjust links to solve pagination
page_num = 1
# category_urls = [link[:-10] + f'page-{page_num}' + link[-5:] for link in category_urls]

category_links = []
for link in category_urls:
    try:
        response = requests.get(link)
        soup = BeautifulSoup(response.content, "lxml")
        soup.find('li', {'class':'current'}).text.split('of')[1].strip()
        category_links.append(link[:-10] + f'page-{page_num}' + link[-5:])
    except:
        category_links.append(link)

In [6]:
# Loop through each category and scrape the pagination details
for category_url in category_links:
    response = requests.get(category_url)
    soup = BeautifulSoup(response.content, "lxml")
    
    # Get the category name
    category = soup.find('h1').text.strip()
    
    try:
        total_pages = int(soup.find('li', {'class':'current'}).text.split('of')[1].strip())
        print(f'{category} has {total_pages} Pages', end='  ')
    except:
        total_pages = 1
        print(f'{category} has {total_pages} Pages', end='  ')


Travel has 1 Pages  Mystery has 2 Pages  Historical Fiction has 2 Pages  Sequential Art has 4 Pages  Classics has 1 Pages  Philosophy has 1 Pages  Romance has 2 Pages  Womens Fiction has 1 Pages  Fiction has 4 Pages  Childrens has 2 Pages  Religion has 1 Pages  Nonfiction has 6 Pages  Music has 1 Pages  Default has 8 Pages  Science Fiction has 1 Pages  Sports and Games has 1 Pages  Add a comment has 4 Pages  Fantasy has 3 Pages  New Adult has 1 Pages  Young Adult has 3 Pages  Science has 1 Pages  Poetry has 1 Pages  Paranormal has 1 Pages  Art has 1 Pages  Psychology has 1 Pages  Autobiography has 1 Pages  Parenting has 1 Pages  Adult Fiction has 1 Pages  Humor has 1 Pages  Horror has 1 Pages  History has 1 Pages  Food and Drink has 2 Pages  Christian Fiction has 1 Pages  Business has 1 Pages  Biography has 1 Pages  Thriller has 1 Pages  Contemporary has 1 Pages  Spirituality has 1 Pages  Academic has 1 Pages  Self Help has 1 Pages  Historical has 1 Pages  Christian has 1 Pages  Suspen

First, let's look at the re.findall(r'\d+', string) part.
This uses the re.findall() function from the re module in Python to find all non-overlapping matches of one or more digits (\d+) in the given string string. 
The result is a list of all the digit strings that were found in the original string, in the order that they appear.

For example, in the given string "Travel has 1 Pages Mystery has 2 Pages", re.findall(r'\d+', string) would return the list ['1', '2'].

In [7]:
import re

# The given string that contains numbers to be summed
string = 'Travel has 1 Pages  Mystery has 2 Pages  Historical Fiction has 2 Pages  Sequential Art has 4 Pages  Classics has 1 Pages  Philosophy has 1 Pages  Romance has 2 Pages  Womens Fiction has 1 Pages  Fiction has 4 Pages  Childrens has 2 Pages  Religion has 1 Pages  Nonfiction has 6 Pages  Music has 1 Pages  Default has 8 Pages  Science Fiction has 1 Pages  Sports and Games has 1 Pages  Add a comment has 4 Pages  Fantasy has 3 Pages  New Adult has 1 Pages  Young Adult has 3 Pages  Science has 1 Pages  Poetry has 1 Pages  Paranormal has 1 Pages  Art has 1 Pages  Psychology has 1 Pages  Autobiography has 1 Pages  Parenting has 1 Pages  Adult Fiction has 1 Pages  Humor has 1 Pages  Horror has 1 Pages  History has 1 Pages  Food and Drink has 2 Pages  Christian Fiction has 1 Pages  Business has 1 Pages  Biography has 1 Pages  Thriller has 1 Pages  Contemporary has 1 Pages  Spirituality has 1 Pages  Academic has 1 Pages  Self Help has 1 Pages  Historical has 1 Pages  Christian has 1 Pages  Suspense has 1 Pages  Short Stories has 1 Pages  Novels has 1 Pages  Health has 1 Pages  Politics has 1 Pages  Cultural has 1 Pages  Erotica has 1 Pages  Crime has 1 Pages  '

# Use regular expressions to find all non-overlapping matches of one or more digits in the string
numbers = re.findall(r'\d+', string)

# Convert the list of digit strings to a list of integers and add them up
sum_numbers = sum(map(int, numbers))

# Print the total sum of all the numbers in the string
print(sum_numbers)

80


In [8]:
# Create an empty list to store the scraped data and pagination start
title = []
price = []
star = []
category = []
link = []

# Loop through each category and scrape the book details
for category_url in category_links:
    if 'page' in category_url:
        page_num = 1
        while True:
            category_url = category_url[:-6] + f'{page_num}' + category_url[-5:]
            result = requests.get(category_url)
            src = result.content
            soup = BeautifulSoup(src, "lxml")
            total_pages = int(soup.find('li', {'class':'current'}).text.split('of')[1].strip())
            
            categorys = soup.find('h1').text.strip()
            titles = soup.find_all('h3')
            prices = soup.find_all('p', {'class':'price_color'})
            stars = soup.find_all("article", class_="product_pod")
            links = soup.find_all('h3')

            for i in range(len(titles)):
                title.append(titles[i].find('a').get('title'))
                price.append(prices[i].text)
                star.append(stars[i].select_one("p.star-rating").get("class")[1])
                category.append(categorys)
                link.append('https://books.toscrape.com/catalogue/' + links[i].find('a').get('href'))
            
            page_num += 1
            
            
            if (page_num > total_pages):
                print(f'Done Scraping {categorys} From {page_num-1} Pages')
                break
    else:
        result = requests.get(category_url)
        src = result.content
        soup = BeautifulSoup(src, "lxml")
        categorys = soup.find('h1').text.strip()
        titles = soup.find_all('h3')
        prices = soup.find_all('p', {'class':'price_color'})
        stars = soup.find_all("article", class_="product_pod")
        links = soup.find_all('h3')

        for i in range(len(titles)):
            title.append(titles[i].find('a').get('title'))
            price.append(prices[i].text)
            star.append(stars[i].select_one("p.star-rating").get("class")[1])
            category.append(categorys)
            link.append('https://books.toscrape.com/catalogue/' + links[i].find('a').get('href'))
        print(f'Done Scraping {categorys}')

with open('Books.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Name', 'Price', 'Rating', 'Category', 'Link'])
        for i in range(len(title)):
            writer.writerow([title[i], price[i], star[i], category[i], link[i]])
pd.read_csv('Books.csv')

Done Scraping Travel
Done Scraping Mystery From 2 Pages
Done Scraping Historical Fiction From 2 Pages
Done Scraping Sequential Art From 4 Pages
Done Scraping Classics
Done Scraping Philosophy
Done Scraping Romance From 2 Pages
Done Scraping Womens Fiction
Done Scraping Fiction From 4 Pages
Done Scraping Childrens From 2 Pages
Done Scraping Religion
Done Scraping Nonfiction From 6 Pages
Done Scraping Music
Done Scraping Default From 8 Pages
Done Scraping Science Fiction
Done Scraping Sports and Games
Done Scraping Add a comment From 4 Pages
Done Scraping Fantasy From 3 Pages
Done Scraping New Adult
Done Scraping Young Adult From 3 Pages
Done Scraping Science
Done Scraping Poetry
Done Scraping Paranormal
Done Scraping Art
Done Scraping Psychology
Done Scraping Autobiography
Done Scraping Parenting
Done Scraping Adult Fiction
Done Scraping Humor
Done Scraping Horror
Done Scraping History
Done Scraping Food and Drink From 2 Pages
Done Scraping Christian Fiction
Done Scraping Business
Done 

,Name,Price,Rating,Category,Link
0,It's Only the Himalayas,£45.17,Two,Travel,https://books.toscrape.com/catalogue/../../../...
1,Full Moon over Noah’s Ark: An Odyssey to Mount...,£49.43,Four,Travel,https://books.toscrape.com/catalogue/../../../...
2,See America: A Celebration of Our National Par...,£48.87,Three,Travel,https://books.toscrape.com/catalogue/../../../...
3,Vagabonding: An Uncommon Guide to the Art of L...,£36.94,Two,Travel,https://books.toscrape.com/catalogue/../../../...
4,Under the Tuscan Sun,£37.33,Three,Travel,https://books.toscrape.com/catalogue/../../../...
...,...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,£52.65,Four,Politics,https://books.toscrape.com/catalogue/../../../...
996,Equal Is Unfair: America's Misguided Fight Aga...,£56.86,One,Politics,https://books.toscrape.com/catalogue/../../../...
997,Amid the Chaos,£36.58,One,Cultural,https://books.toscrape.com/catalogue/../../../...
998,Dark Notes,£19.19,Five,Erotica,https://books.toscrape.com/catalogue/../../../...


<details><summary>Done By</summary>
<center> Ahmed NasrElDin </center> 
</details>
